# 0. Imports

In [3]:
import config as creds
import pandas as pd

# also possible using sqlalchemy
import psycopg2


## 0.1 Helper functions

In [14]:
def connect():
    
    # set up a connection to the postgres server.
    conn_string = ("host="+ creds.PGHOST +\
                   " port="+ "5432" +\
                   " dbname="+ creds.PGDATABASE +\
                   " user=" + creds.PGUSER+ \
                   " password="+ creds.PGPASSWORD)
    
    conn = psycopg2.connect(conn_string)

    # create a cursor object
    # used to perform queries
    cursor = conn.cursor()
    
    return conn, cursor


# 1.0 Schema query

In [12]:
# connecting to DB
conn, cursor = connect()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute(query_schema)
record = (cursor.fetchall())
print(record)
cursor.close()
conn.close()


#cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg|sql)';")
#print (cursor.fetchall())

Connected!


[('pg_toast',),
 ('pg_temp_1',),
 ('pg_toast_temp_1',),
 ('pg_catalog',),
 ('information_schema',),
 ('public',),
 ('pa004',),
 ('pg_temp_4',),
 ('pg_toast_temp_4',),
 ('pa005',),
 ('pg_temp_9',),
 ('pg_toast_temp_9',)]

# 2.0 Table query

In [17]:
#connecting to DB
conn, cursor = connect()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname ='pa004'
"""

cursor.execute(query_tables)
record = (cursor.fetchall())
print(record)
cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0 Collect data using psycopg2

In [18]:
# connecting to DB
conn, cursor = connect()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

cursor.execute(query_tables_users)
record = (cursor.fetchall())
print(record)
cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [23]:
# not good because it is necessary to enter columns names manually
# to improve this, we can make data collection using pandas
pd.DataFrame(record).head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


# 4.0 Collect data using pandas

## 4.1 Using pandas.io.sql

In [24]:
import pandas.io.sql as psql

In [32]:
conn, cursor = connect()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

# the columns names are obtained when using pandas to collect data
display(psql.read_sql(query_tables_users, conn).head())

cursor.close()
conn.close()


,id,gender,age,region_code,policy_sales_channel
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


## 4.2 Using only pandas

In [33]:
conn, cursor = connect()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

# the columns names are obtained when using pandas to collect data
display(pd.read_sql(query_tables_users, conn).head())

cursor.close()
conn.close()

,id,gender,age,region_code,policy_sales_channel
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


In [ ]:
#select *
#from
#	pa004.users u inner join pa004.vehicle v on(u.id=v.id)
#				  inner join pa004.insurance i on(u.id = i.id)
#limit 10